In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from datetime import datetime as dt
from casestudy.update import *

In [3]:
from see19 import CaseStudy

%load_ext autoreload
%autoreload 2

In [10]:
### Setup Exception Logger

# Setup logger    
LOG_PATH = '/Users/spindicate/Documents/programming/zooscraper/casestudy/update/update_logs/'
filename = 'update-{}.log'.format(dt.now().strftime('%Y-%m-%d'))
logfile = LOG_PATH + filename

In [ ]:
exc_logger = ExceptionLogger(logfile)

In [3]:
### Run Updates

# Loop through update functions and log any errors 
for func in update_funcs:
    timeout = 10*60 if func.__name__ == 'update_msmts' else 60
    wrapfunc = exc_logger.wrap('exception', timeout=timeout)(func)
    wrapfunc(create=True)

Running get_italy...
Running get_braz...
Running get_US...
Running create_rest...
Running update_austests...
Running update_cadtests...
Running update_ustests...
Running update_resttests...
Running update_strindex...
Running update_gmobi...


/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Aruba AW
Cote d'Ivoire 
Cape Verde CV
Laos LA
North Macedonia MK
Réunion RE
Tajikistan TJ
Running update_amobi...
Instantiating Chrome WebDriver...
Running update_msmts...


Running update_pollutants...


In [4]:
### Test

make_baseframe = exc_logger.wrap('critical')(make_baseframe)
baseframe = make_baseframe()

test_region_consistency = exc_logger.wrap('critical')(test_region_consistency)
test_region_consistency(baseframe)

for count_type in CaseStudy.COUNT_TYPES:
    test_notnas = exc_logger.wrap('exception')(test_notnas)
    test_notnas(baseframe, count_type)

factors_with_dmas = ['strindex']
kwargs = {'factors': CaseStudy.ALL_FACTORS, 'interpolate_method': {'method': 'linear'}}
casestudy = CaseStudy(baseframe, **kwargs)

test_duplicate_dates = exc_logger.wrap('exception')(test_duplicate_dates)
test_duplicate_dates(casestudy)
test_duplicate_days = exc_logger.wrap('exception')(test_duplicate_days)
test_duplicate_days(casestudy)
test_negative_days = exc_logger.wrap('exception')(test_negative_days)
test_negative_days(casestudy)

Running make_baseframe...
initializing
adding fatalities
adding cases
adding tests
adding pollutants
adding measurements
backfill time-static data
adding strindex
adding Google Mobility
adding Apple Mobility
adding Causes of Death
adding Travel popularity
adding GDP
COMPLETE
Running test_region_consistency...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Exception(' ...Failing Regions: [407]')
Running test_duplicate_dates...
Running test_duplicate_days...
Running test_negative_days...


In [ ]:
### Send email and, If no critical errors, push to git 
with open(logfile, 'r') as f:
    log_text = f.read()

if 'CRITICAL' in log_text:
    log_email(logfile, critical=True)
else:
    
    baseframe = make_baseframe(save=True)

In [4]:
baseframe = make_baseframe()

initializing
adding fatalities
adding cases
adding tests
adding pollutants
adding measurements
backfill time-static data
adding strindex
adding Google Mobility
adding Apple Mobility
adding Causes of Death
adding Travel popularity
adding GDP
COMPLETE


In [5]:
note = ''
update_readme(note)    

/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript', 'application/vnd.bokehjs_load.v0+json']) is not able to be represented.
  mimetypes=output.keys())
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript', 'application/vnd.bokehjs_exec.v0+json']) is not able to be represented.
  mimetypes=output.keys())


In [11]:
# git_push()
log_email(logfile)    

In [19]:
repo = Repo(config('ROOTPATH'))

In [ ]:
repo.git.add('.')
repo.git.commit(m='')

In [21]:
repo.git.push('origin')

"Branch 'master' set up to track remote branch 'master' from 'origin'."

In [7]:
assert style in ['dataset', 'testset', 'rm_only']

adds = [config('SEE19PATH') + item for item in ['dataset/', 'latest_dataset.txt', 'README.md']]
m = 'update dataset'
if style == 'testset':
    adds += [config('SEE19PATH') + item for item in ['testset', 'latest_testset.txt']]
    m += ' and testset'
elif style == 'rm_only':
    adds = config('SEE19PATH') + 'README.md'
    m = 'update README'

if style in ['dataset', 'testset']:
    m += ' {}'.format(dt.now().strftime('%Y-%m-%d %H:%M:%S'))

repo.git.add(*adds)
repo.git.commit(m=m)
repo.git.subtree('push', 'see19', 'master', prefix='casestudy/see19/')
repo.remote(name='origin').push()

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git push --porcelain origin
  stderr: 'fatal: The current branch master has no upstream branch.
To push the current branch and set the remote as upstream, use

    git push --set-upstream origin master
'